In [69]:
import nyc_db
from importlib import reload
reload(nyc_db);
from nyc_db import DB
import pandas as pd
import sqlite3
import dbga
from collections import defaultdict as dd
import json

In [43]:
my_sql = dd(dict)

In [44]:
my_sql['setup'] = []

In [46]:
conn = sqlite3.connect('databases/nyc_health.db')
curs = conn.cursor()

In [47]:
data = pd.read_csv('nyc_rest_health_inspections.csv')
data.head(2)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION
0,40511702,NOTARO RESTAURANT,MANHATTAN,635,SECOND AVENUE,10016.0,2126863400,Italian,06/15/2015,Violations were cited in the following area(s).,02B,Hot food item not held at or above 140Âº F.
1,40511702,NOTARO RESTAURANT,MANHATTAN,635,SECOND AVENUE,10016.0,2126863400,Italian,11/25/2014,Violations were cited in the following area(s).,20F,Current letter grade card not posted.


---
1) Get a list of all restaurants (by camis) who have been cited for a violation containing the words 'vermin', 'mice', or 'rats'.

Have your query return the following columns:
- camis
- dba
- viol_id
- viol_desc
- A count of the number of times they have been cited for each violation

Order the results with the most violations on top.

In [48]:
sql1 = '''
SELECT camis, dba, viol_desc, viol_id, count(*) as NumViol
FROM tInsp
JOIN tViol USING(viol_id)
JOIN tRest USING(camis)
WHERE viol_desc LIKE "%vermin%" OR viol_desc LIKE "%mice%" OR viol_desc LIKE "%rats%"
GROUP BY camis, viol_desc
ORDER BY NumViol DESC
;'''
pd.read_sql(sql1, conn)

,camis,dba,viol_desc,viol_id,NumViol
0,40861669,NEW VICTORY RESTAURANT,Facility not vermin proof. Harborage or condit...,08A,15
1,41430594,1 STOP PATTY SHOP,Evidence of mice or live mice present in facil...,04L,14
2,50033122,PARTY WELL REST & ORIENTAL BAKERY,Facility not vermin proof. Harborage or condit...,08A,14
3,40861669,NEW VICTORY RESTAURANT,Evidence of mice or live mice present in facil...,04L,13
4,41309387,NEW TACO EXPRESS,Evidence of mice or live mice present in facil...,04L,13
...,...,...,...,...,...
29033,50068060,TASTE OF PUNJAB,Facility not vermin proof. Harborage or condit...,08A,1
29034,50068098,SUBWAY,Facility not vermin proof. Harborage or condit...,08A,1
29035,50068305,SONRISA AZUL,Facility not vermin proof. Harborage or condit...,08A,1
29036,50068537,JOE & THE JUICE COLOMBUS,Evidence of mice or live mice present in facil...,04L,1


---
2) For each different cuisine type, determine the most commonly cited violation **excluding viol_id 10F** (this is by far the most common violation, and not very "gross").

Have your query return:

- cuisine
- viol_id
- viol_desc
- Number of violations for their most common violation (except 10F)

Limit those results to cuisine types where the max number of violations is greater than 1000.

Order the results in ascending order by the cuisine type.

In [50]:
sql2 = '''
WITH
ViolBYCuisineViol_id as 
(
    SELECT count(*) as NumViol, cuisine, viol_id, viol_desc
    FROM tInsp
    JOIN tViol USING(viol_id)
    JOIN tRest USING(camis)
    GROUP BY cuisine, viol_id
)

SELECT cuisine, viol_id, viol_desc, max(NumViol) as NumViol
FROM (ViolBYCuisineViol_id)
WHERE viol_id NOT LIKE "%10f%"
GROUP BY cuisine
HAVING NumViol > 1000
ORDER BY cuisine ASC
;'''
pd.read_sql(sql2, conn)

,cuisine,viol_id,viol_desc,NumViol
0,American,08A,Facility not vermin proof. Harborage or condit...,8573
1,Bakery,08A,Facility not vermin proof. Harborage or condit...,1355
2,CafÃ©/Coffee/Tea,08A,Facility not vermin proof. Harborage or condit...,1308
3,Caribbean,08A,Facility not vermin proof. Harborage or condit...,1697
4,Chinese,08A,Facility not vermin proof. Harborage or condit...,4438
5,Italian,08A,Facility not vermin proof. Harborage or condit...,1575
6,Japanese,08A,Facility not vermin proof. Harborage or condit...,1360
7,"Latin (Cuban, Dominican, Puerto Rican, South &...",08A,Facility not vermin proof. Harborage or condit...,2065
8,Mexican,08A,Facility not vermin proof. Harborage or condit...,1586
9,Pizza,08A,Facility not vermin proof. Harborage or condit...,2111


---

3) When the action description contains the word 'closed', what are the top 5 associated violations?


Have your query return:

- action_desc
- viol_desc
- total number of violations

Order the results with the largest number of violations on top

In [52]:
sql3 = '''
SELECT action_desc, viol_desc, count(*) as NumViol
FROM tInsp
JOIN tViol USING(viol_id)
JOIN tAction USING(action_id)
WHERE action_desc LIKE "%closed%"
GROUP BY viol_desc
ORDER BY NumViol DESC
LIMIT 5
;'''
pd.read_sql(sql3, conn)

,action_desc,viol_desc,NumViol
0,Establishment Closed by DOHMH. Violations wer...,Facility not vermin proof. Harborage or condit...,1656
1,Establishment Closed by DOHMH. Violations wer...,Evidence of mice or live mice present in facil...,1148
2,Establishment Closed by DOHMH. Violations wer...,Live roaches present in facility's food and/or...,715
3,Establishment Closed by DOHMH. Violations wer...,Cold food item held above 41Âº F (smoked fish ...,691
4,Establishment Closed by DOHMH. Violations wer...,Non-food contact surface improperly constructe...,640


---

4) Determine which restaurant has been closed the most (action description contains 'closed').

For that camis, have your query return:

- camis
- dba
- viol_desc
- total number of violations per violation type

Include all violations for that camis (not just the ones they were closed for)

Order the results with the most common violation on top

In [54]:
sql4 = '''
WITH
MostClosedCamis as
(
    SELECT count(*) as MostClosed, camis, dba
    FROM tInsp
    JOIN tViol USING(viol_id)
    JOIN tAction USING(action_id)
    JOIN tRest USING(camis)
    WHERE action_desc LIKE "%closed%"
    GROUP BY camis
    ORDER BY MostClosed DESC
    LIMIT 1
)


SELECT camis, dba, viol_desc, count(*) as NumViol
FROM tInsp
JOIN tViol USING(viol_id)
JOIN tAction USING(action_id)
JOIN tRest USING(camis)
WHERE camis LIKE (SELECT camis FROM MostClosedCamis)
GROUP BY viol_id
ORDER BY NumViol DESC
;'''
pd.read_sql(sql4, conn)

,camis,dba,viol_desc,NumViol
0,40922818,EL TAPATIO MEXICAN RESTAURANT,Non-food contact surface improperly constructe...,8
1,40922818,EL TAPATIO MEXICAN RESTAURANT,Facility not vermin proof. Harborage or condit...,7
2,40922818,EL TAPATIO MEXICAN RESTAURANT,Live roaches present in facility's food and/or...,7
3,40922818,EL TAPATIO MEXICAN RESTAURANT,Plumbing not properly installed or maintained;...,5
4,40922818,EL TAPATIO MEXICAN RESTAURANT,Evidence of rats or live rats present in facil...,5
5,40922818,EL TAPATIO MEXICAN RESTAURANT,Cold food item held above 41Âº F (smoked fish ...,4
6,40922818,EL TAPATIO MEXICAN RESTAURANT,Failure to comply with an Order of the Board o...,3
7,40922818,EL TAPATIO MEXICAN RESTAURANT,The original nutritional fact labels and/or in...,3
8,40922818,EL TAPATIO MEXICAN RESTAURANT,Food Protection Certificate not held by superv...,3
9,40922818,EL TAPATIO MEXICAN RESTAURANT,Hot food item not held at or above 140Âº F.,3


---

5) Group all restaurants by their DBA, and computer the average number of violations per location.

For example, if there were 100 restaurants with the dba "STARBUCKS", and they had a total of 123 violations, then the average number of violations per location would be 123/100=1.23.

Have your query return:
- dba
- Number of locations
- Number of violations
- Avg violations per location

Order the results with the largest number of locations on top

_Watch out for integer division!_

In [56]:
sql5 = '''

SELECT dba, TotalRestNum, NumViol, 1.0*NumViol/TotalRestNum as AvgViolsPerLoc
FROM (
    SELECT count(DISTINCT(camis)) as TotalRestNum, count(camis) as NumViol, dba
    FROM tInsp
    JOIN tViol USING(viol_id)
    JOIN tAction USING(action_id)
    JOIN tRest USING(camis)
    GROUP BY dba
    ORDER by TotalRestNum DESC
    )

;'''
pd.read_sql(sql5, conn)

,dba,TotalRestNum,NumViol,AvgViolsPerLoc
0,DUNKIN' DONUTS,410,4247,10.358537
1,SUBWAY,352,3729,10.593750
2,STARBUCKS,281,2130,7.580071
3,MCDONALD'S,206,2778,13.485437
4,"DUNKIN' DONUTS, BASKIN ROBBINS",110,1462,13.290909
...,...,...,...,...
20242,$1 PIZZA,1,8,8.000000
20243,#1 SABOR LATINO RESTAURANT,1,55,55.000000
20244,#1 Natural Juice Bar,1,6,6.000000
20245,#1 GARDEN CHINESE,1,27,27.000000


---

6) For restaurants (by DBA) with only a single location (i.e., DBA with exactly 1 associated CAMIS) and more than 10 total violations, what is their top violation, and how many times have they been cited for it?

Note that a restaurant might have been cited 1 time each for 10 different types of violations, and 2 times for another violation. That would be 12 total violations, which counts as more than 10.

**Limit your results** to cases where their top violation occurred greater than or equal to 15 times.

Have your query return:

- camis
- dba
- viol_desc
- number of violations

Order your results by number of violations, descending.

In [58]:
#20247 rows of dba's total
#18908 restuarnats with only one camis
#11620 restaurnats with 10 or more viols with only one camis

In [59]:
sql6 = '''
WITH
RestList as
(
SELECT camis, dba, count(DISTINCT(camis)) as GetRest, viol_id
FROM tInsp
JOIN tViol USING(viol_id)
JOIN tRest USING(camis)
GROUP BY dba
HAVING GetRest = 1
),
TotalViolNum as
(
SELECT camis, dba, count(*) as NumViol
FROM tInsp
JOIN tViol USING(viol_id)
JOIN tRest USING(camis)
WHERE camis IN (SELECT camis FROM RestList)
GROUP BY dba
HAVING NumViol > 10
ORDER BY NumViol
),

NoMore as
(
SELECT camis, dba, count(*) as NumViol, viol_desc
FROM tInsp
    JOIN tViol USING(viol_id)
    JOIN tRest USING(camis)
WHERE camis IN (SELECT camis FROM TotalViolNum)
GROUP BY dba, viol_id
ORDER BY NumViol DESC
)

SELECT camis, dba, viol_desc, Max(NumViol) as NumViol
FROM NoMore
GROUP BY dba
HAVING NumViol >= 15
ORDER BY NumViol DESC

;'''
pd.read_sql(sql6, conn)

,camis,dba,viol_desc,NumViol
0,40861669,NEW VICTORY RESTAURANT,Facility not vermin proof. Harborage or condit...,15


---

7) Considering all possible combinations of violation/action, how many of those combinations **do not occur** in the data?

Have your query return just the total number.

In [61]:
sql7 = '''
WITH
AllCombs as
  (
   SELECT viol_id, action_id
   FROM tViol
   JOIN tAction
  ),
HasOccurred as
  (
   SELECT DISTINCT viol_id, action_id
   FROM tInsp
  )
SELECT count(*)
FROM(
SELECT AllCombs.action_id, AllCombs.viol_id, HasOccurred.action_id
FROM AllCombs
LEFT JOIN HasOccurred USING(action_id, viol_id)
WHERE HasOccurred.action_id IS NULL
)
;'''
pd.read_sql(sql7, conn)

,count(*)
0,221


---

8) Return the action and violation descriptions associated with the previous question.

Have your query return:

- action_desc
- viol_desc

Order your results in ascending order by viol_desc

In [63]:
sql8 = '''
WITH
AllCombs as
  (
  SELECT viol_id, action_id
  FROM tViol
  JOIN tAction
  ),
HasOccurred as
  (
  SELECT DISTINCT viol_id, action_id
  FROM tInsp
  )
SELECT action_desc, viol_desc
FROM AllCombs
LEFT JOIN HasOccurred USING(action_id, viol_id)
JOIN tViol USING(viol_id)
JOIN tAction USING(action_id)
WHERE HasOccurred.action_id IS NULL
ORDER BY viol_desc ASC
;'''
pd.read_sql(sql8, conn)

,action_desc,viol_desc
0,Establishment Closed by DOHMH. Violations wer...,''''No Smoking and/or 'Smoking Permitted sig...
1,Establishment re-opened by DOHMH,''''No Smoking and/or 'Smoking Permitted sig...
2,Establishment re-closed by DOHMH,''''No Smoking and/or 'Smoking Permitted sig...
3,Establishment re-closed by DOHMH,''''Wash hands sign not posted at hand wash f...
4,No violations were recorded at the time of thi...,''''Wash hands sign not posted at hand wash f...
...,...,...
216,Establishment re-opened by DOHMH,"Whole frozen poultry or poultry breasts, other..."
217,Establishment re-closed by DOHMH,"Whole frozen poultry or poultry breasts, other..."
218,No violations were recorded at the time of thi...,"Whole frozen poultry or poultry breasts, other..."
219,Establishment re-closed by DOHMH,Wiping cloths soiled or not stored in sanitizi...


---

9) For all restaurants (by camis) that have been cited for 90 or more violations, what is their top violation?

Have your query return:
- camis
- dba
- viol_id
- viol_desc
- Number of violations (for the top violation only)

Order the results in ascending order by dba


In [65]:
sql9 = '''
WITH 
NumViol_90_ as 
  (
  SELECT count(*) as NumViol, camis, dba
  FROM tInsp
  JOIN tRest USING(camis)
  JOIN tViol USING(viol_id)
  GROUP BY camis
  HAVING NumViol >= 90
  ),
ViolByCamis as
  (
  SELECT camis, dba, viol_id, viol_desc, count(*) as NumViol
  FROM tInsp
  JOIN tRest USING(camis)
  JOIN tViol USING(viol_id)
  WHERE camis IN (SELECT camis FROM NumViol_90_)
  GROUP BY viol_id, camis
  )
SELECT camis, dba, viol_id, viol_desc, max(NumViol) as NumViol
FROM ViolByCamis
GROUP BY camis
ORDER BY dba ASC
;'''
pd.read_sql(sql9, conn)

,camis,dba,viol_id,viol_desc,NumViol
0,50001880,GANG SAN DEUL,02B,Hot food item not held at or above 140Âº F.,10
1,40965177,LA GATA GOLOSA #2,06C,Food not protected from potential source of co...,8
2,41683816,MAX BAKERY & RESTAURANT,04M,Live roaches present in facility's food and/or...,12
3,50033122,PARTY WELL REST & ORIENTAL BAKERY,08A,Facility not vermin proof. Harborage or condit...,14
4,41459659,T. K. KITCHEN,08A,Facility not vermin proof. Harborage or condit...,13


In [67]:
conn.close()